In [3]:
import os
import sys
import openpyxl
import pandas as pd

from config import *



'D:\\Dev\\Project\\Poe_Tyr_chs\\temp_output'

In [4]:
# df merge test

tdf = pd.read_excel(os.path.join(TEMP_DIR, 'Tyranny_trans_result.xlsx'), index_col='index').fillna('')
tdf['m_DefaultText'] = '' 
tdf['m_FemaleText'] = ''

In [5]:
from lib import *
# 多翻译合并策略，ali为主，没翻译的，用3dm的补充
def trans_merge(df):
    group = {}
    for result in translated_result_list[TYPE_NAME]:
        d_key = '[{group_name}]DefaultText'.format(group_name = result['group_name'])
        f_key = '[{group_name}]FemaleText'.format(group_name = result['group_name'])
        
        group[result['group_name']] = {}
        group[result['group_name']]['d'] = d_key
        group[result['group_name']]['f'] = f_key
    print(group)
    
    for i, row in df.iterrows():
        mer = ''
        fer = ''
        l = group['ali']['d']
        r = group['3dm']['d']
        lf = group['ali']['f']
        rf = group['3dm']['f']

        # d
        if row[l] != 'NaN':
            if has_ch(str(row[r])) and not has_ch(str(row[l])):
                mer = row[r]
            else:
                mer = row[l]
        # f
        if row[lf] and row[lf] !='' and row[lf] != 'NaN':
            if has_ch(str(row[rf])) and not has_ch(str(row[lf])):
                fer = row[rf]
            else:
                fer = row[lf]
        df.ix[i, 'm_DefaultText'] = mer
        df.ix[i, 'm_FemaleText'] = fer

    print('hebin')
    return df

tdf = trans_merge(tdf)

{'3dm': {'d': '[3dm]DefaultText', 'f': '[3dm]FemaleText'}, 'ali': {'d': '[ali]DefaultText', 'f': '[ali]FemaleText'}}
hebin


In [7]:
tdf.ix[25826]

[ali]DefaultText    Bank Node 5
[ali]FemaleText                
[3dm]DefaultText         堤岸节点 5
[3dm]FemaleText                
m_DefaultText            堤岸节点 5
m_FemaleText                   
Name: conversations\08_vendrienswell\08_cv_scarletchoruscamp_dog_barks_5, dtype: object

In [8]:
# 用合并好的生成再和string生成双语对比
string_excel = string_result_output_filename
sdf = pd.read_excel(string_excel)
sdf.index = [sdf['Name'] + '_' + sdf['ID'].astype(str)]


In [9]:
sdf['m_DefaultText'] = tdf['m_DefaultText']
sdf['m_FemaleText'] = tdf['m_FemaleText']

# temp_merge = os.path.join(TEMP_DIR,'tdf.xlsx')
# tdf[['mer']].to_excel(temp_merge)

In [116]:
def gloss_patch(Series, **args): 
    gloss_file = os.path.join(STORAGE_DIR, 'tyr_gloss.xlsx')
    gloss_list = pd.read_excel(gloss_file).fillna('')

    en_key = 'en'
    chs_key = 'chs'
    # code key
    reg = '\[url=(.*?)\]'
    gloss_list['code'] = gloss_list[en_key].str.extract(reg,expand=False)
    gloss_list['chs_code'] = gloss_list[chs_key].str.extract('\[url=.*?\](.*?)(?=\[/url\])',expand=False)
    gloss_list = gloss_list.fillna('')
    
    if 'type' not in args:
        # 替换为 [url=glossary:{en]{中文}[/url]
        for i,row in gloss_list.iterrows():
            if row[chs_key] != '':
                print('rep: ' + row[chs_key])
                re = '(\[url='+row['code']+'?[^\]]*\].*?\[/url\])'
                Series = Series.str.replace(re, row[chs_key])
    else:
        # 替换为 [url=glossary: {中文}]{中文}[/url]
        for i, row in gloss_list.iterrows():  
            if row[chs_key] != '':
                re = '(\[url='+row['code']+'?[^\]]*\].*?\[/url\])'
                repl = '[url=glossary:{chs}]{chs}[/url]'.format(chs = row['chs_code'])
                Series = Series.str.replace(re, repl)
                
    return Series


In [130]:
# gloss_file = os.path.join(STORAGE_DIR, 'tyr_gloss.xlsx')
# gloss_list = pd.read_excel(gloss_file).fillna('')
# gloss_list
sdf['m_DefaultText'] = gloss_patch(sdf['m_DefaultText'], type=1)

In [167]:
sdf.ix[36475]['m_DefaultText']

'发射一根光芒万丈的箭矢。经过的所有敌人都将受到伤害并且会[url=glossary:目盲]目盲[/url]。任何遭到打击的祸灵都会触发一次对区域内所有祸灵造成秘法伤害的小型爆炸。效果与死弓的声望成正比。'

In [142]:
hcv = pd.read_excel(os.path.join(STORAGE_DIR, 'tyr_en-chs.xlsx'))
hcv.index = [hcv['Name'] +'_'+ hcv['ID'].astype(str)]

In [164]:
# kov.drop('Custom',axis=1)
kov

,Name,ID,DefaultText,FemaleText,Custom,Custom_female,m_DefaultText,m_FemaleText,Custom_c
conversations\00_stronghold\00_cv_act2_skypillar_defense_arri_1,conversations\00_stronghold\00_cv_act2_skypill...,1,"""What should we do, Fatebinder?""",NaN,NaN,NaN,"""我们该怎么办，缚命官？""",,"""What should we do, Fatebinder?"""
conversations\00_stronghold\00_cv_act2_skypillar_defense_arri_2,conversations\00_stronghold\00_cv_act2_skypill...,2,"""This must be dealt with. Let's go speak with ...",NaN,NaN,NaN,"""必须解决这件事，我们去和他们谈谈。""",,"""This must be dealt with. Let's go speak with ..."
conversations\00_stronghold\00_cv_act2_skypillar_defense_arri_3,conversations\00_stronghold\00_cv_act2_skypill...,3,"""Let them wait. I'm not ready yet.""",NaN,NaN,NaN,"""让他们等着吧，我还没准备好。""",,"""Let them wait. I'm not ready yet."""
conversations\00_stronghold\00_cv_act2_skypillar_defense_arri_4,conversations\00_stronghold\00_cv_act2_skypill...,4,"""There are too many. The walls will only hold ...",NaN,NaN,NaN,"""太多了，墙壁无法支持太久。""",,"""There are too many. The walls will only hold ..."
conversations\00_stronghold\00_cv_act2_skypillar_defense_ceveus_1,conversations\00_stronghold\00_cv_act2_skypill...,1,"""What should we do, Fatebinder?""",NaN,NaN,NaN,"""我们该怎么办，缚命官？""",,"""What should we do, Fatebinder?"""
conversations\00_stronghold\00_cv_act2_skypillar_defense_ceveus_2,conversations\00_stronghold\00_cv_act2_skypill...,2,"""This must be dealt with. Let's go speak with ...",NaN,NaN,NaN,"""必须解决这件事，我们去和他们谈谈。""",,"""This must be dealt with. Let's go speak with ..."
conversations\00_stronghold\00_cv_act2_skypillar_defense_ceveus_3,conversations\00_stronghold\00_cv_act2_skypill...,3,"""Let them wait. I'm not ready yet.""",NaN,NaN,NaN,"""让他们等着吧，我还没准备好。""",,"""Let them wait. I'm not ready yet."""
conversations\00_stronghold\00_cv_act2_skypillar_defense_ceveus_4,conversations\00_stronghold\00_cv_act2_skypill...,4,"""There are too many. The walls will only hold ...",NaN,NaN,NaN,"""太多了，墙壁无法支持太久。""",,"""There are too many. The walls will only hold ..."
conversations\00_stronghold\00_cv_act2_skypillar_defense_rayth_1,conversations\00_stronghold\00_cv_act2_skypill...,1,"""What should we do, Fatebinder?""",NaN,NaN,NaN,"""我们该怎么办，缚命官？""",,"""What should we do, Fatebinder?"""
conversations\00_stronghold\00_cv_act2_skypillar_defense_rayth_2,conversations\00_stronghold\00_cv_act2_skypill...,2,"""This must be dealt with. Let's go speak with ...",NaN,NaN,NaN,"""必须解决这件事，我们去和他们谈谈。""",,"""This must be dealt with. Let's go speak with ..."


In [156]:
# kov[(kov['Custom_c'].isnull()) & (kov['DefaultText'].notnull())]

In [165]:
sdffile = os.path.join(TEMP_DIR, 'sdf.xlsx')

# openpyxl 为 pandas添加样式并输出
from openpyxl.utils.dataframe import dataframe_to_rows
font = openpyxl.styles.Font(name='微软雅黑',
                            size=10,
                            color='FF000000')
# 220 230 241
fill = openpyxl.styles.PatternFill("solid", fgColor="DDDDDD")

wb = openpyxl.Workbook()
ws = wb.active
x = 0
# 转换为list
for r in dataframe_to_rows(kov, index=False, header=True):
#     print(r)
    x += 1
#     if x % 2 == 0:
#         print(x)
    ws.append(r)

rownumb = 0
for ro in ws:
    rownumb += 1
    for c in ro:
        c.font = font
    if rownumb % 2 == 0:
        for c in ro:
            c.fill = fill
            
for cell in ws[1]:
    cell.style = 'Pandas'

wb.save(sdffile)

In [98]:
# pandas style
import numpy as np
tempdata = {'state':['ohio','ohio','ohio','nev','nev'],'year':[2000,2001,2002,2003,2004],'pop':[1.5,1.3,1.7,2.0,3.5]}
tempdf = pd.DataFrame(tempdata)
tempdf['y2'] = [2000, 2001, 2008, 2006, 2004]

def color_negative_red(val):
    color = 'red' if val < 2 else 'black'
    return 'color: %s' % color

tempdf.style.applymap(color_negative_red, subset=pd.IndexSlice[0: , ['pop']])

In [135]:
def ups(val):
    print(type(val))
    print(val[[0]])
    return val

tempdf[['year','y2']].apply(ups)
# tempdf.ix[0]

NameError: name 'tempdf' is not defined